## Maine Senate overview script

In [2]:
import numpy as np
import pandas as pd
import requests
import config
import os
import json
import pygsheets
from pandas.io.json import json_normalize
import time

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_˜

In [132]:
### SENATE CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Senate candidates
cand_state = 'ME'
period='2020'
parameters = {'election_year':period
            ,'state':cand_state
            ,'office':'S'
            ,'api_key':config.api_key}

#Requests candidate info
r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()

#Locates and sets Committee ID from 'principal_committees' sub-array
#Output list of IDs
comm_ids=json_normalize(data=r_cands['results'],record_path='principal_committees')['committee_id'].tolist()
comm_ids

['C00710087',
 'C00314575',
 'C00709899',
 'C00703413',
 'C00706739',
 'C00683193',
 'C00723411',
 'C00709279',
 'C00712323',
 'C00694364']

In [97]:
## INDIVIDUAL DONATIONS TO MAINE SENATE CAMPAIGNS ##
#Initialize dataframe collector for itemized contribs
cycle='2020'
idfs=[]
udfs=[]
commid=0

#Initialize query dictionary
itemdict = {
    'per_page':'100'
    ,'sort':'contribution_receipt_date'
    ,'api_key':config.api_key
    ,'is_individual':'true'
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_contribution_receipt_date':[]
    ,'committee_id':comm_ids[commid]
}

#Dict for unitemized contributions
unitemdict = {
'api_key':config.api_key
,'cycle':cycle
,'per_page':'100'
,'committee_id':comm_ids[commid]
}

#Page through results for each committee id
for x in range(0,len(comm_ids)):
    
    u_r = requests.get('https://api.open.fec.gov/v1/committee/'+comm_ids[commid]+'/totals',params=unitemdict).json()
    udf = json_normalize(u_r['results'])
    udfs.append(udf)
    
    #Get first itemized payload for a candidate
    itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    #Print itemdict to validate
    print(itemdict)
    
    #Last page variables
    while itemr['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        idf = json_normalize(itemr['results'])
        idfs.append(idf)
                
        #Assign last_index and date values, update itemdict
        last_index=pd.to_numeric(itemr['pagination']['last_indexes']['last_index'])
        last_date=itemr['pagination']['last_indexes']['last_contribution_receipt_date']
        #Update dictionary with new indices
        itemdict.update([('last_index',last_index)
                        ,('last_contribution_receipt_date',last_date)])

        #Get next payload with updated dict
        itemr = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    commid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    itemdict.update([('committee_id',comm_ids[commid])
                     ,('last_index',[])
                     ,('last_contribution_receipt_date',[])])
    
    unitemdict.update([('committee_id',comm_ids[commid])])

{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00710087'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00314575'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_receipt_date': [], 'committee_id': 'C00709899'}
{'per_page': '100', 'sort': 'contribution_receipt_date', 'api_key': 'egxSs7endLz5xMuoprm5zfVZCeoyeZbO5D6HFzJz', 'is_individual': 'true', 'two_year_transaction_period': '2020', 'last_index': [], 'last_contribution_rece

IndexError: list index out of range

In [103]:
# Concatenate all dfs
itemdf=pd.concat(idfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

udf=pd.concat(udfs,sort=False,ignore_index=True)
udf=udf.drop_duplicates()

In [111]:
itemdf.to_csv('sample_df.csv')

In [112]:
#ITEMIZED DATA CLEANING#
itemdf['contributor_zip'] = itemdf['contributor_zip'].str[:5]

#Table slimming
cols = [
    'committee.name'
    ,'committee.party_full'
    ,'contribution_receipt_amount'
    ,'contribution_receipt_date'
    ,'entity_type'
    ,'contributor_city'
    ,'contributor_state'
    ,'contributor_street_1'
    ,'contributor_zip'
    ,'contributor_employer'
    ,'contributor_name'
    ,'contributor_occupation'
    ,'fec_election_type_desc'
    ,'load_date'
    ,'pdf_url'
    ,'transaction_id'
]

itemdf=itemdf[cols]
itemdf

,committee.name,committee.party_full,contribution_receipt_amount,contribution_receipt_date,entity_type,contributor_city,contributor_state,contributor_street_1,contributor_zip,contributor_employer,contributor_name,contributor_occupation,fec_election_type_desc,load_date,pdf_url,transaction_id
0,COLLINS FOR SENATOR,REPUBLICAN PARTY,250.0,2019-01-01T00:00:00,IND,SAINT PETERSBURG,FL,15019 MADEIRA WAY,33738,RETIRED,"BENTLEY, HENRY",RETIRED,PRIMARY,2019-04-09T02:42:36.656000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A1F99550E176249C7AFC
1,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,TUSCALOOSA,AL,4114 CITATION CIR,35406,SELF,"BRECKINRIDGE, FLINT",CONSULTANT,PRIMARY,2019-04-09T02:42:36.677000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A66BF5ABC54F943FA94E
2,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,SILOAM SPRINGS,AR,16735 OLD HIGHWAY 68,72761,ARVEST BANK,"GIBBS, CLIFFORD",MANAGEMENT,PRIMARY,2019-04-09T02:42:36.764000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AE9C7DC4C7855441A843
3,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,HILLSBORO BEACH,FL,953 HILLSBORO MILE,33062,RETIRED,"PARELL, JOHN",RETIRED,PRIMARY,2019-04-09T02:42:36.843000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AB3531B9E6A664583BE4
4,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,POULSBO,WA,10750 OGLE RD NE,98370,RETIRED,"REED, FRANK",RETIRED,PRIMARY,2019-04-09T02:42:36.852000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A8BDD8FFE4FE34C7DAA7
5,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,POULSBO,WA,10750 OGLE RD NE,98370,RETIRED,"REED, FRANK",RETIRED,PRIMARY,2019-04-09T02:42:36.852000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AA2E9430417814DCAB3B
6,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-01T00:00:00,IND,DALLAS,TX,4412 LARCHMONT ST,75205,HTPN,"SCHUMACHER, JOHN",PHYSICIAN,PRIMARY,2019-04-09T02:42:36.867000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,A0950552B2BCE4148A89
7,COLLINS FOR SENATOR,REPUBLICAN PARTY,100.0,2019-01-02T00:00:00,IND,LOS GATOS,CA,15965 LANCASTER RD,95030,ALL SEASONS ROOFING,"GORSHTEYN, VLAD",CONSTRUCTION EXECUTIVE,PRIMARY,2019-04-09T02:42:36.769000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AB2FD1F7334D440D0917
8,COLLINS FOR SENATOR,REPUBLICAN PARTY,250.0,2019-01-02T00:00:00,IND,BILOXI,MS,8051 CRESTWOOD CV,39532,INDEPENDENT CONTRACTOR,"O'CONNOR, RICHARD",IT SECURITY MGR,PRIMARY,2019-04-09T02:42:36.838000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,ADFE3250EAA894F028D3
9,COLLINS FOR SENATOR,REPUBLICAN PARTY,50.0,2019-01-03T00:00:00,IND,GAINESVILLE,FL,309 NW 24TH ST,32607,RETIRED,"FOX, JOHN",RETIRED,PRIMARY,2019-04-09T02:42:36.744000+00:00,http://docquery.fec.gov/cgi-bin/fecimg/?201904...,AC9527181AD7C477EB4D


In [113]:
#Create DataFrame with columns to match itemized table
unitemdf=[]
unitemdf = pd.DataFrame(columns=cols)

## Select data for unitemized df ##
unitemdf[['committee.name'
        ,'committee.party_full'
        ,'contribution_receipt_amount'
        ,'contribution_receipt_date'
        ,'fec_election_type_desc']] = udf[['committee_name'
                                        ,'party_full'
                                        ,'individual_unitemized_contributions'
                                        ,'coverage_end_date'
                                        ,'last_report_type_full']]

#Label as unitemized
unitemdf['contributor_name'] = 'Unitemized individual contributions'
unitemdf['entity_type'] = 'IND'

In [114]:
#Union Itemized and Unitemized contributions
ind_df = pd.concat([itemdf,unitemdf],sort=False,ignore_index=True)

In [115]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
ind_df.to_csv(cwd+'/data/individual-senate-contributions.csv')

In [117]:
#Write out summary file from totals endpoint
udf.to_csv(cwd+'/senate-financial-summary.csv')

In [131]:
#QA - compare itemized values to totals
qadf = itemdf.groupby(['committee.name']).agg({'contribution_receipt_amount':['sum']})
qadf

totaldf = udf[['committee_name'
             ,'individual_itemized_contributions'
             ,'individual_unitemized_contributions']]

qadf = qadf.merge(totaldf, left_on='committee.name',right_on='committee_name')

/Users/Darren/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(contribution_receipt_amount, sum)",committee_name,individual_itemized_contributions,individual_unitemized_contributions
0,4498439.77,COLLINS FOR SENATOR,3853089.77,158832.34
1,3300.00,DEREK FOR U.S. SENATE,3550.00,5062.80
2,8760.00,FRIENDS OF BRE KIDMAN,9560.00,2796.02
3,20550.00,JON TREACY FOR SENATE,20815.00,0.00
4,2381778.37,SARA GIDEON FOR MAINE,2269803.37,1782220.27
5,82678.95,SWEET FOR SENATE,82878.95,100166.55


In [40]:
# #Google Credentials
# gc = pygsheets.authorize(service_file=cwd+'/../me-congress-2020-creds.json')

# #Select sheet and worksheet
# sh = gc.open('maine-senate-2020')
# # sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
# wks = sh[0]

# #Clear sheet before load
# wks.clear(start='A1',fields='*')

# #Write contribs dataframe to sheet
# wks.set_dataframe(df_cull,(1,1))

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/12bHf1qEtKtGGje0a3lBJJ8r-IvYcs3yl5jxbP6jNMCo/values/Individual%20itemized%21A3334%3AP6667?valueInputOption=USER_ENTERED&alt=json returned "Range ('Individual itemized'!A3334:P6667) exceeds grid limits. Max rows: 3333, max columns: 120">